The aim of this nb is to populate a postgresdatabase with Forex data from Darwinex

In [ ]:
import os

%pip install psycopg2
import psycopg2

PASSWORD=""

def open_connection(database="darwinex3"):
    #establishing the connection
    conn = psycopg2.connect(
            database=database,
            user='postgres', password=PASSWORD, host='127.0.0.1', port= '5432'
    )
    conn.autocommit = True
    return conn

In [ ]:
# Add code to connect Darwinex FTP and download the missing days


In [ ]:
def create_extension_timescaledb(dbname):
    conn = open_connection(database=dbname)
    sql = '''create extension if not exists timescaledb;'''
    cursor.execute(sql)
    print("Configuring timescaledb")
    conn.commit()
    conn.close()

In [ ]:
# This depends on the timeseriesdb timescale.com
# installation docs at https://docs.timescale.com/install/latest/

def create_table(pair,side,database,drop=False):
    conn = open_connection(database=database)
    cursor = conn.cursor()
    table_name = f"d_{pair}_{side}"
    # Avoid droping tables by mistake
    if drop:
        cursor.execute(f"DROP TABLE IF EXISTS {table_name}")        
        print(f"Delete created successfully........for {table_name}")
    
    # TODO: If table does not exists
    sql =f'''CREATE TABLE d_{pair}_{side}(
       TIMESTAMP TIMESTAMP NOT NULL,
       DATETIME VARCHAR(15),
       {side} FLOAT NOT NULL,
       VOL FLOAT
    )'''
    cursor.execute(sql)    
    print(f"Table {table_name} created successfully........")    
    # create hypertable
    sql=f'''select create_hypertable('{table_name}','timestamp');'''
    cursor.execute(sql)
    print(f"Table {table_name} created hypertable........")
    conn.close()


In [ ]:
# TODO

database = "darwinex3"
pairs=["usdchf"]

for pair in pairs:
    side="ask"
    create_table(pair,side,database)
    side="bid"
    create_table(pair,side,database)


In [ ]:
from datetime import datetime

def get_datetime(fname):
    date = fname.split('_')[2]
    time =  fname.split('_')[3].split('.')[0]
    #print(date,time)
    return date+'_'+time

def unzip_log_file(full_path):
    import gzip    
    registers=''
    try:
        with gzip.open(full_path,'rb') as f:
            file_content = f.read()
        registers = file_content.decode('ascii').split("\n")        
    except OSError:
        print(OSError.errno)
        print(f"Error unzipping {full_path} | ")
        return None
    return registers
    
def launch_sql_insert_query(values, table_name, side):    
    sql=None
    try:
        jvalues = ",".join(values)     
        sql = f'''INSERT INTO {table_name}(timestamp,datetime,{side},vol) VALUES {jvalues}'''    
        conn = open_connection()
        cursor = conn.cursor()
        cursor.execute(sql)        
        conn.commit()
        conn.close()
    except:
        #print("sql query",sql)
        print(f"Problem found inserting into table {table_name}")
        return "KO"

def collect_csv_registers(registers,datehour):
    # collect from csv resisters    
            
    values = []
    for reg in registers:
        if reg == '':
            continue
            
        sreg = reg.split(',')
        
        #print(sreg)
        
        try:
            timestamp = datetime.fromtimestamp(float(float(sreg[0])/1000))
            
            price     = float(sreg[1]) 
            vol       = int(float(sreg[2]))
        
            sreg_for_db = f"('{timestamp}','{datehour}','{price}','{vol}')"
        
            #print(sreg_for_db)
        
            values.append(sreg_for_db)
        except ValueError:
            print("Problem converting data from registers",sreg)
    return values
    
def insert_into_db(side,pair,fname,path):

    full_path = path+"\\"+fname
    
    registers = unzip_log_file(full_path)

    if registers == None:
        return "KO problem found extrating resigers"
    
    datehour = get_datetime(fname)
    
    values = collect_csv_registers(registers,datehour)

    table_name = f"d_{pair}_{side}"
    
    launch_sql_insert_query(values, table_name, side)    
        
    return "OK commit "+fname


def insert_bulk_pair(root,pair,side, resume_at=None):
    import os
    path = os.path.join(root,pair.upper())    
    lfiles = os.listdir(path)
    assert len(lfiles) > 0
    
    index_to_resume = None
    if resume_at != None:
        index_to_resume = resume_at.get(pair,None)

    #print(index_to_resume)
    for index, fname in enumerate(lfiles):
     
        if index_to_resume!=None and index_to_resume > index:
            #print(f"{index} skip file {fname}")
            continue            
        
        if not fname.endswith(".log.gz"):
            continue
        
        if side.upper() not in fname:
            continue
        
        is_ok = insert_into_db(side,pair,fname,path)
        if(is_ok =="KO"):
            print("something went wrong",index)
            break
        print(datetime.now(), index, is_ok)
        
        
def multiple_inserts(root, pairs, sides, resume_at=None):
    for pair in pairs: 
        for side in sides:
            insert_bulk_pair(root,pair,side, resume_at)

In [ ]:
root = 'C:\\Users\\ruben\\darwinex_data'
resume_at = {}
resume_at["eurgbp"] = 15449 # Resume ask at 22880
pairs  = ['eurgbp','usdchf'] 
sides  = ['ask','bid'] 
multiple_inserts(root,pairs, sides, resume_at)

In [ ]:
# this is the way to ingest ms into postgres
#datetime.fromtimestamp(float('1506891900032')/1000)

#conn = open_connection()
#fname='USDCAD_ASK_2020-02-28_19.log.gz'
#pair='usdcad'
#insert_into_db(side,pair,fname,conn)

#fname=path+'\\USDCAD_ASK_2020-02-28_19.log.gz'
#with gzip.open(full_path,'rb') as f:
#    file_content = f.read()
#print(file_content)